In [4]:
%pylab inline
import pandas as pd
import os
DATA_DIR = '/Users/Will/Data/lending_club/'

Populating the interactive namespace from numpy and matplotlib


In [5]:
os.listdir(DATA_DIR)

['.DS_Store',
 'LoanStats3a.csv',
 'LoanStats3b.csv',
 'LoanStats3c.csv',
 'LoanStats3d.csv']

In [259]:
df = pd.DataFrame()
for csv in os.listdir(DATA_DIR)[1:]:
    df = df.append(pd.read_csv(DATA_DIR + csv, header=True))

/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [261]:
important_columns = ['total_pymnt', 'zip_code', 'member_id', 'id', 'loan_amnt', 'int_rate',
                    'installment', 'emp_length', 'home_ownership', 'grade', 'sub_grade', 'emp_title',
                    'issue_d', 'loan_status', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'pymnt_plan',
                     'inq_last_6mths', 'dti', 'revol_util', 'mths_since_last_record', 'mths_since_last_delinq',
                     'pub_rec', 'revol_bal', 'open_acc', 'collections_12_mths_ex_med', 'delinq_2yrs',
                     'earliest_cr_line',  'last_credit_pull_d']

In [262]:
lc = df[important_columns]

In [128]:
other_columns = ['last_pymnt_d', 'last_pymnt_amnt']

In [263]:
mean(df.total_pymnt - df.total_pymnt_inv)

50.786928010183928

Earliest CR line is a good proxy for age.

# Type Casting

In [300]:
lc = lc[:-2] # The last 2 rows of the CSV arent proper rows

In [301]:
lc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646387 entries, 0 to 180099
Data columns (total 30 columns):
total_pymnt                   646387 non-null float64
zip_code                      646387 non-null int64
member_id                     646387 non-null float64
id                            646387 non-null int64
loan_amnt                     646387 non-null float64
int_rate                      646387 non-null float64
installment                   646387 non-null float64
emp_length                    615969 non-null float64
home_ownership                646387 non-null category
grade                         646387 non-null category
sub_grade                     646387 non-null int64
emp_title                     609363 non-null object
issue_d                       646387 non-null datetime64[ns]
loan_status                   646387 non-null category
annual_inc                    646383 non-null float64
verification_status           646387 non-null int64
purpose                 

In [274]:
lc['zip_code'] = lc['zip_code'].str.replace('x', '')

In [275]:
lc.zip_code

0         860
1         309
2         606
3         917
4         972
5         852
6         280
7         900
8         958
9         774
10        853
11        913
12        245
13        606
14        951
15        641
16        921
17        067
18        890
19        921
20        770
21        335
22        799
23        917
24        900
25        605
26        103
27        150
28        326
29        564
         ... 
180072    925
180073    325
180074    209
180075    176
180076    339
180077    597
180078    412
180079    308
180080    840
180081    138
180082    282
180083    550
180084    871
180085    021
180086    330
180087    333
180088    891
180089    325
180090    286
180091    471
180092    956
180093    531
180094    430
180095    432
180096    791
180097    905
180098    072
180099    010
180100    378
180101    331
Name: zip_code, dtype: object

In [282]:
lc.zip_code = lc.zip_code.astype(int)

In [283]:
lc = lc[~lc.id.str.contains('[A-z]', na=False)] #Remove ids with words

In [284]:
lc['id'] = lc.id.astype(int)

In [285]:
lc.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a'], dtype=object)

In [286]:
lc.emp_length = lc.emp_length.replace({'10+ years': 20, '< 1 year': 0, '1 year': 1})
lc.emp_length = lc.emp_length.replace({'{} years'.format(i): i for i in range(1,10)})
lc.emp_length = lc.emp_length.replace({'n/a': np.nan})

In [287]:
lc.int_rate = lc.int_rate.str.replace('%', '').astype(float) / 100

In [288]:
lc.home_ownership = lc.home_ownership.astype('category')

In [289]:
# Change the subgrades to numeric values

unique_subgrades = sorted(lc.sub_grade.unique())
lc.sub_grade = lc.sub_grade.replace({'{}'.format(i):
                      unique_subgrades.index(i) for i in unique_subgrades})

In [290]:
lc['grade'] = lc['grade'].astype('category')

In [291]:
lc.issue_d = pd.to_datetime(lc.issue_d)

In [292]:
lc.loan_status = lc.loan_status.astype('category')

In [293]:
lc.verification_status = lc.verification_status.replace({'VERIFIED - income': 1,
                                'VERIFIED - income source': 1, 'not verified': 0 })

In [294]:
lc.purpose = lc.purpose.astype('category')

In [295]:
lc.addr_state = lc.addr_state.astype('category')

In [296]:
lc = lc.drop('pymnt_plan', axis=1)

In [297]:
lc['revol_util'] = lc.revol_util.str.replace("%", '').astype(float) / 100

In [298]:
lc.earliest_cr_line = pd.to_datetime(lc.earliest_cr_line)

In [299]:
lc.last_credit_pull_d = pd.to_datetime(lc.last_credit_pull_d)

In [304]:
import pickle
with open('cleaned_df.pkl', 'wb') as picklefile:
    pickle.dump(lc, picklefile)

# Feature Engineering

Feature | Why
-- | --
Ratio of income to installment payment | might suggest overpayment amount
Ratio of income to all debt including installment | The previous but better.

In [310]:
lc[['installment', 'revol_bal']].mean()

installment      435.056483
revol_bal      16545.933747
dtype: float64

In [315]:
lc.columns

Index(['total_pymnt', 'zip_code', 'member_id', 'id', 'loan_amnt', 'int_rate',
       'installment', 'emp_length', 'home_ownership', 'grade', 'sub_grade',
       'emp_title', 'issue_d', 'loan_status', 'annual_inc',
       'verification_status', 'purpose', 'addr_state', 'inq_last_6mths', 'dti',
       'revol_util', 'mths_since_last_record', 'mths_since_last_delinq',
       'pub_rec', 'revol_bal', 'open_acc', 'collections_12_mths_ex_med',
       'delinq_2yrs', 'earliest_cr_line', 'last_credit_pull_d',
       'ratio_inc_debt'],
      dtype='object')

In [316]:
lc['ratio_inc_debt'] = (lc.loan_amnt + lc.revol_bal) / lc.annual_inc

In [317]:
lc.ratio_inc_debt.describe()

count    646383.000000
mean          0.458122
std           0.257535
min           0.001072
25%           0.277250
50%           0.418118
75%           0.592603
max          10.706455
Name: ratio_inc_debt, dtype: float64

In [322]:
lc['ratio_inc_installment'] = lc.installment / (lc.annual_inc / 12)

In [323]:
lc.ratio_inc_installment.describe()

count    646383.000000
mean          0.080259
std           0.040342
min           0.000241
25%           0.049286
50%           0.075108
75%           0.107037
max           0.541710
Name: ratio_inc_installment, dtype: float64

In [332]:
lc['ratio_mth_inc_all_payments'] = (lc.installment + lc.revol_bal * .02) / (lc.annual_inc / 12)

In [333]:
lc.ratio_mth_inc_all_payments.describe()

count    646383.000000
mean          0.137122
std           0.071193
min           0.000393
25%           0.085831
50%           0.126881
75%           0.177213
max           2.644211
Name: ratio_mth_inc_all_payments, dtype: float64

# List of features

dti